
# Storytime


Before running or if using a different scenario, the user needs to  
make the below changes.  
* Open the scenario 
* Set the animation time & time step
* In createfolders() decide how the files should be structured  

The goal in this script is to simplify generating eoir synthetic data  
This is done in a few steps 

## _main()_

main() kicks everything off. Using the scenario time, it calculates the  
number of total frames between start and stop. This will be the number  
of times eoirdata() steps through the scenario. The loop jumps to the  
beginning of the scenario then cycles through the following cloud and  
atmospheric parameters.   

|Clouds| . |Atmosphere|
|:-----|:-:|---------:|
|Off   | . |Off       |
|On    | . |Off       |
|On    | . |Simple       |
|On    | . |MODTRAN   |

Once the parameters are set, it dives into eoirdata()

## _eoirdata()_  

eoirdata() uses connect commands to generate the bitmap and the  
raw data. In the same line, the data is stored into it's respective  
folder (see createfolders()). This function then steps forward in time  
and does the whole thing again. The range, or number of iterations is   
set by the number of frames. Once it reaches the end time the function  
jumps out. 

## _createfolders()_  
Create the folders that eoirdata() will save to. This only needs to be run once.  

## _Misc_

attach to scenario PID  

In the future decide if the created folders should be saved to a list  
and read into the connect commands for robustness. The purpose would be to  
stop from having to make manual updates  

Optionally, set a command to define the clouds:     
root.ExecuteCommand('EOIR */ CloudData SetValue MyConstDataSet Altitude 12    
PercentCloudCoverConst 1 PercentCloudCoverFile   
"C:\ProgramData\AGI\STK 12\STKData\VO\Clouds\clouds_2020126.100000_2020127.160000.cld  
EmissivityConst .5 TemperatureConst 233 ForwardScatterConst .1)  

clean up file name, move ftype earlier in the statement  
make relevant updates


# Connect Command Reference

Atmosphere Key
* 0 - off.
* 1 - Simple atmosphere.
* 2 - MODTRAN atmosphere.

Atmosphere Simple Parameters
* Aerosol Models: Rural
* Visibility: 25.00 km
* Humidity: 45.8 %
    
Cloud Key
* On - turns on clouds.
* Off - turns off clouds.


In [11]:
import os 
import csv
import itertools
from datetime import timedelta
from datetime import datetime
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# Open EOIR_Clouds_AirToAir.sc
# Connect to the scenario
# Get the IAgStkObjectRoot interface 

stk      = STKDesktop.AttachToApplication()  
root     = stk.Root                           
scenario = root.CurrentScenario 

# Set the scenario animation properties
animation               = scenario.Animation
animation.StartTime     = '5 May 2020 11:41:30.000'
animation.EnableAnimCycleTime = True
animation.AnimCycleTime = '5 May 2020 11:46:00.000'
animation.AnimCycleType = AgEScEndLoopType.eLoopAtTime
animation.AnimStepValue = 0.5


In [ ]:
def createfolders():

    data    = ['raw','processed'] #  the state of the data
    sensor  = ['LWIR','Visible']  #  names of the sensors in the scenario
    ftype   = ['bitmap','txt']    #  exported data format

    cld     = ['Off','On','On','On']
    atm     = [0,0,1,2]

    path = "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python/"
    newdir = os.mkdir("{}output/".format(path))

    for (a,b,c) in itertools.product(data,ftype,sensor): # to do: put ftype earlier in the name
        for (x,y) in itertools.zip_longest(cld,atm):
            name = "{}_{}_{}_Cld{}_Atm{}".format(a, b, c, x, y)
            #print("{}output/{}".format(path,name))
            os.mkdir("{}output/{}".format(path,name))

createfolders()

In [12]:
def eoirdata(x,y,z):

    command_bmp_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python/output/raw_bitmap_LWIR")
    command_txt_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python/output/raw_txt_LWIR")
    command_bmp_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python/output/raw_bitmap_Visible")
    command_txt_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python/output/raw_txt_Visible")
    
    for i in range(z):
        # print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        # print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        # print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        # print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.StepForward()



In [13]:
def main():

    d1 = datetime.strptime(animation.StartTime,    "%d %b %Y %H:%M:%S.%f")
    d2 = datetime.strptime(animation.AnimCycleTime, "%d %b %Y %H:%M:%S.%f")
    k  = int(timedelta.total_seconds(d2 - d1)/animation.AnimStepValue)
    #k  = 1 # quick run for testing

    cld     = ['On'] #['Off','On','On','On']
    atm     = [2]    #[0,0,1,2]

    for (i,j) in itertools.zip_longest(cld,atm):
        root.Rewind()
        root.ExecuteCommand("EOIR */ SetAtmosphere Clouds {}".format(i))
        root.ExecuteCommand("EOIR */ SetAtmosphere Mode {}".format(j))
        eoirdata(i,j,k)

if __name__ == '__main__':
    main()  